<a href="https://colab.research.google.com/github/wILLIEWILLYWILLIe/hw_1/blob/main/mlhw1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ML2025 Homework 1 - Retrieval Augmented Generation with Agents

## Environment Setup

First, we will mount your own Google Drive and change the working directory.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from re import M
# Change the working directory to somewhere in your Google Drive.
# You could check the path by right clicking on the folder.
%cd /content
!pwd
!ls
%cd drive/MyDrive/Colab_Notebooks/hw1
!pwd

/content
/content
drive  sample_data
/content/drive/MyDrive/Colab_Notebooks/hw1
/content/drive/MyDrive/Colab_Notebooks/hw1


In this section, we install the necessary python packages and download model weights of the quantized version of LLaMA 3.1 8B. Also, download the dataset. Note that the model weight is around 8GB. If you are using your Google Drive as the working directory, make sure you have enough space for the model.

In [ ]:
!python3 -m pip install --no-cache-dir llama-cpp-python==0.3.4 --extra-index-url https://abetlen.github.io/llama-cpp-python/whl/cu122
!python3 -m pip install googlesearch-python bs4 charset-normalizer requests-html lxml_html_clean

from pathlib import Path
if not Path('./Meta-Llama-3.1-8B-Instruct-Q8_0.gguf').exists():
    !wget https://huggingface.co/bartowski/Meta-Llama-3.1-8B-Instruct-GGUF/resolve/main/Meta-Llama-3.1-8B-Instruct-Q8_0.gguf
if not Path('./public.txt').exists():
    !wget https://www.csie.ntu.edu.tw/~ulin/public.txt
if not Path('./private.txt').exists():
    !wget https://www.csie.ntu.edu.tw/~ulin/private.txt

Looking in indexes: https://pypi.org/simple, https://abetlen.github.io/llama-cpp-python/whl/cu122
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 445.2/445.2 MB 289.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 206.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.1/201.1 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.2/144.2 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.4/107.4 kB 12.1 MB/s eta 0:00:00
  Attempting uninstall: websockets
    Found existing installation: websockets 14.2
    Uninstalling websockets-14.2:
      Successfully uninstalled websockets-14.2
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.3.0
    Uninstalling urllib3-2.3.0:
      Successfully uninstalled urllib3-2.3.0
ERROR: pip's dep

In [ ]:
import torch
if not torch.cuda.is_available():
    raise Exception('You are not using the GPU runtime. Change it first or you will suffer from the super slow inference speed!')
else:
    print('You are good to go!')

You are good to go!


## Prepare the LLM and LLM utility function

By default, we will use the quantized version of LLaMA 3.1 8B. you can get full marks on this homework by using the provided LLM and LLM utility function. You can also try out different LLM models.

In the following code block, we will load the downloaded LLM model weights onto the GPU first.
Then, we implemented the generate_response() function so that you can get the generated response from the LLM model more easily.

You can ignore "llama_new_context_with_model: n_ctx_per_seq (16384) < n_ctx_train (131072) -- the full capacity of the model will not be utilized" warning.

In [96]:
from llama_cpp import Llama
token_max_len = 16384
token_max_len = 32000
# Load the model onto GPU
llama3 = Llama(
    "./Meta-Llama-3.1-8B-Instruct-Q8_0.gguf",
    verbose=False,
    n_gpu_layers=-1,
    n_ctx=token_max_len,    # This argument is how many tokens the model can take. The longer the better, but it will consume more memory. 16384 is a proper value for a GPU with 16GB VRAM.
)

def generate_response(_model: Llama, _messages: str) -> str:
    '''
    This function will inference the model with given messages.
    '''
    _output = _model.create_chat_completion(
        _messages,
        stop=["<|eot_id|>", "<|end_of_text|>"],
        max_tokens=512,    # This argument is how many tokens the model can generate, you can change it and observe the differences.
        temperature=0,      # This argument is the randomness of the model. 0 means no randomness. You will get the same result with the same input every time. You can try to set it to different values.
        repeat_penalty=2.0,
    )["choices"][0]["message"]["content"]
    return _output

llama_new_context_with_model: n_ctx_per_seq (32000) < n_ctx_train (131072) -- the full capacity of the model will not be utilized


## Search Tool

The TA has implemented a search tool for you to search certain keywords using Google Search. You can use this tool to search for the relevant **web pages** for the given question. The search tool can be integrated in the following sections.

In [97]:
from typing import List
from googlesearch import search as _search
from bs4 import BeautifulSoup
from charset_normalizer import detect
import asyncio
from requests_html import AsyncHTMLSession
import urllib3
urllib3.disable_warnings()

async def worker(s:AsyncHTMLSession, url:str):
    try:
        header_response = await asyncio.wait_for(s.head(url, verify=False), timeout=10)
        if 'text/html' not in header_response.headers.get('Content-Type', ''):
            return None
        r = await asyncio.wait_for(s.get(url, verify=False), timeout=10)
        return r.text
    except:
        return None

async def get_htmls(urls):
    session = AsyncHTMLSession()
    tasks = (worker(session, url) for url in urls)
    return await asyncio.gather(*tasks)

async def search(keyword: str, n_results: int=3) -> List[str]:
    '''
    This function will search the keyword and return the text content in the first n_results web pages.

    Warning: You may suffer from HTTP 429 errors if you search too many times in a period of time. This is unavoidable and you should take your own risk if you want to try search more results at once.
    The rate limit is not explicitly announced by Google, hence there's not much we can do except for changing the IP or wait until Google unban you (we don't know how long the penalty will last either).
    '''
    keyword = keyword[:100]
    # First, search the keyword and get the results. Also, get 2 times more results in case some of them are invalid.
    results = list(_search(keyword, n_results * 2, lang="zh", unique=True))
    # Then, get the HTML from the results. Also, the helper function will filter out the non-HTML urls.
    results = await get_htmls(results)
    # Filter out the None values.
    results = [x for x in results if x is not None]
    # Parse the HTML.
    results = [BeautifulSoup(x, 'html.parser') for x in results]
    # Get the text from the HTML and remove the spaces. Also, filter out the non-utf-8 encoding.
    results = [''.join(x.get_text().split()) for x in results if detect(x.encode()).get('encoding') == 'utf-8']
    # Return the first n results.
    return results[:n_results]

def check_len(myinput : str) -> str:
    max_length = token_max_len
    if len(myinput) >= max_length :
      available_space = max_length - 4
      myinput = myinput[:available_space] + "..."
    return myinput


## Test the LLM inference pipeline

In [ ]:
# You can try out different questions here.
test_question='請問誰是 Taylor Swift？'
messages = [
    {"role": "system", "content": "你是 LLaMA-3.1-8B，是用來回答問題的 AI。使用中文時只會使用繁體中文來回問題。"},    # System prompt
    {"role": "user", "content": test_question}, # User prompt
]

print(generate_response(llama3, messages))

泰勒絲（Taylor Swift）是一位美國歌手、詞曲作家和音樂製作人。她出生於1989年，來自田納西州。她的音乐风格从乡村摇滚发展到流行搖擺，並且她被誉为当代最成功的女艺人的之一。

泰勒絲早期在鄉郊小鎮演唱會時開始發展音樂事業，她推出了多張專輯，包括《Taylor Swift》、《Fearless》，以及後來更為知名的大熱作品——「1989」。她以其寫作風格、歌曲創造力和表現能力而聞name。

泰勒絲的音乐风味从乡村摇滚到流行搖擺，並且她的专辑《Fearless》、《Speak Now》，以及後來更為知名的大熱作品——「1989」、「Reputation」，都取得了巨大的成功。


## Agents

The TA has implemented the Agent class for you. You can use this class to create agents that can interact with the LLM model. The Agent class has the following attributes and methods:
- Attributes:
    - role_description: The role of the agent. For example, if you want this agent to be a history expert, you can set the role_description to "You are a history expert. You will only answer questions based on what really happened in the past. Do not generate any answer if you don't have reliable sources.".
    - task_description: The task of the agent. For example, if you want this agent to answer questions only in yes/no, you can set the task_description to "Please answer the following question in yes/no. Explanations are not needed."
    - llm: Just an indicator of the LLM model used by the agent.
- Method:
    - inference: This method takes a message as input and returns the generated response from the LLM model. The message will first be formatted into proper input for the LLM model. (This is where you can set some global instructions like "Please speak in a polite manner" or "Please provide a detailed explanation".) The generated response will be returned as the output.

In [98]:
class LLMAgent():
    def __init__(self, role_description: str, task_description: str, llm:str="bartowski/Meta-Llama-3.1-8B-Instruct-GGUF"):
        self.role_description = role_description   # Role means who this agent should act like. e.g. the history expert, the manager......
        self.task_description = task_description    # Task description instructs what task should this agent solve.
        self.llm = llm  # LLM indicates which LLM backend this agent is using.
    def inference(self, message:str) -> str:
        if self.llm == 'bartowski/Meta-Llama-3.1-8B-Instruct-GGUF': # If using the default one.
            # TODO: Design the system prompt and user prompt here.
            # Format the messsages first.
            messages = [
                # Hint: you may want the agents to speak Traditional Chinese only.
                {"role": "system", "content": f"{self.role_description}使用中文時只會使用繁體中文來回問題。"},
                # Hint: you may want the agents to clearly distinguish the task descriptions and the user messages.
                # A proper seperation text rather than a simple line break is recommended.
                {"role": "user", "content": f"{self.task_description}\n===\n{message}"}
            ]

            return generate_response(llama3, messages)
        else:
            # TODO: If you want to use LLMs other than the given one, please implement the inference part on your own.
            return ""

In [ ]:
role_description = "你是音樂與流行文化的專家，負責提供關於藝人與音樂的準確資訊。"
task_description = "請根據使用者的問題，提供簡潔且準確的回答。"
task_description = ""
agent = LLMAgent(role_description, task_description)

# Test the question
test_question = "請問誰是 Taylor Swift？"
response = agent.inference(test_question)
print(response)

泰勒絲（Taylor Swift）是美國的一位知名流行音樂歌手、詞曲作家和製作人。她出生於1989年12月13日，肯塔基州曼徹斯特。她的父親是一個銀行經理，她的母校曾贏得全美最佳高中獎項。

泰勒絲從小就對唱跳有濃厚興趣，並在2003年的農曆新歲時參加了「卡拉OK」比賽，獲得第一名。她之後開始寫作歌曲並錄製demo帶，以便向音樂公司推銷自己的作品。最終，她與大機器娛樂簽下唱片合約。

泰勒絲的首張專輯《Taylor Swift》於2006年發行，並在美國獲得了商業上的成功。她之後陸續出版多部歌曲和专辑，包括「Fearless」、「Speak Now」，以及與斯考特·布倫南（Scott Borchetta）合作成立自己的唱片公司——泰勒絲共和國。

她的音樂風格從鄉村搖滾逐漸轉變為流行電音，並且她以其對社會和個人問題的深刻描述，以及歌曲中的人物化手法而聞名。許多人認爺，她是當代最具影響力的女性藝術家之一。

泰勒絲也是一位頗受關注的大眾人物，曾經與其他知星進行過公開爭議，並且她對社會和政治問題的表態備受到矚目。她在2020年發行了專輯《Folklore》，並於隔年的「1989世界巡迴演唱會」中宣布退休。


TODO: Design the role description and task description for each agent.

In [99]:
# TODO: Design the role and task description for each agent.

# This agent may help you filter out the irrelevant parts in question descriptions.
question_extraction_agent = LLMAgent(
    role_description="你是問題解析專家，擅長從使用者的輸入中提取核心問題，並確保問題清晰且具體。",
    task_description=(
        "請分析以下輸入，提取核心問題" #並包含問題重要條件，例如地點、人物、年代、區域。\n"
        "若輸入包含多個問題，請選擇最主要的一個。\n"
        "僅回答一句話即可。"
    )
)

# This agent may help you extract the keywords in a question so that the search tool can find more accurate results.
keyword_extraction_agent = LLMAgent(
    role_description="你是關鍵詞萃取專家，負責從問題中提取最適合搜尋的關鍵詞，包括專有名詞、核心概念或動作及地點、",
    task_description="請從以下問題中提取關鍵詞，確保其具體且具代表性，用逗號分隔。避免過於泛泛的詞彙。"
)

# This agent is the core component that answers the question.
qa_agent = LLMAgent(
    role_description="你是問答生成專家，負責根據問題和相關資訊提供準確、簡潔且符合事實的回答。",
    task_description=(
        "請根據以下問題和篩選後的相關資訊，提供準確的回答。\n"
        "若有`相關資訊` 提供了明確的數字（如分數、金額、時間），請直接使用該數據來回答問題。\n"
        "請使用正式且易於理解的繁體中文回答，確保資訊清晰。\n"
        "一定要幫我整理好資料並輸出最好的繁體中文結果，這對我至官重要。"
    )
)

context_filter_agent = LLMAgent(
    role_description="你是內容篩選專家，負責從搜尋結果中最相關的資訊，簡化內容以提高問答效率。",
    task_description=(
        "請根據以下問題，從相關資料中提取最正確的資訊，並用繁體中文回答問題。\n"
        "請確保答案與問題相關，優先保留數字資訊（如分數、金額、時間等）。\n"
        "若出現不同數字，請仔細比對何者為正確的。\n"
        "如果有72數字在內，擷取其數字。\n"
        "若有其他相關資訊或候選答案，也一併輸出。"
    )
)

## RAG pipeline

TODO: Implement the RAG pipeline.

Please refer to the homework description slides for hints.

Also, there might be more heuristics (e.g. classifying the questions based on their lengths, determining if the question need a search or not, reconfirm the answer before returning it to the user......) that are not shown in the flow charts. You can use your creativity to come up with a better solution!

- Naive approach (simple baseline)

    ![](https://www.csie.ntu.edu.tw/~ulin/naive.png)

- Naive RAG approach (medium baseline)

    ![](https://www.csie.ntu.edu.tw/~ulin/naive_rag.png)

- RAG with agents (strong baseline)

    ![](https://www.csie.ntu.edu.tw/~ulin/rag_agent.png)

In [102]:
async def pipeline(question: str) -> str:
    DO_DEBUG = True
    ########################################################
    # TODO: Implement your pipeline.
    # Currently, it only feeds the question directly to the LLM.
    # You may want to get the final results through multiple inferences.
    # Just a quick reminder, make sure your input length is within the limit of the model context window (16384 tokens),
    # You may want to truncate some excessive texts.
    ########################################################
    if DO_DEBUG : print(f'{" "*5}question: {question}')

    # Step 1: Extract the core question
    core_question = question_extraction_agent.inference(question)
    if DO_DEBUG : print(f'{" "*5}core_question: {core_question}')
    if not core_question:
        return "無法解析問題，請重新輸入。"

    # Step 2: Extract keywords from the core question
    keywords = keyword_extraction_agent.inference(question)
    if DO_DEBUG: print(f'{" "*5}key_words: {keywords}')
    if not keywords:
        return "無法提取關鍵詞，請提供更具體的問題。"

    # # Step 3: Retrieve context using the provided search function
    # raw_answer1 = qa_agent.inference(question)
    # raw_search_results = await search(question, n_results=2)
    # raw_search_results2 = await search(keywords, n_results=3)
    # raw_search_results = [raw_answer1] + raw_search_results + raw_search_results2
    # # if DO_DEBUG : print('   raw_result: ', raw_search_results)
    # filter_output_tot = ""
    # if raw_search_results :
    #   for idx, raw_result in enumerate(raw_search_results):
    #     filter_input  = check_len(f"問題：{core_question}\n相關資料: {' '.join(raw_result)}")
    #     filtered_context = context_filter_agent.inference(filter_input)
    #     if not filtered_context: pass
    #     else :
    #       filter_output_tot += filtered_context
    #       if DO_DEBUG :print(f'{" "*5}{idx}-filter_context\n{filtered_context}')
    # if not raw_search_results:
    #     filter_output_tot = "未找到相關資訊，根據已有知識回答。"

    # # Step 5: Generate the final answer
    # final_input = f"問題：{core_question}\n相關資訊: {filter_output_tot}"
    # final_input = check_len(final_input)
    # response = qa_agent.inference(final_input)

    # check_len(response)

    # return response

    # Step 3: Retrieve context using the provided search function
    raw_answer1 = qa_agent.inference(question)
    raw_answer1 = ""
    raw_search_results  = []
    # raw_search_results += await search(core_question, n_results=1)
    raw_search_results += await search(keywords, n_results=3)
    for _ in raw_search_results :
      raw_answer1 += _
    final_input = check_len(f"問題：{core_question}\n相關資料: {raw_answer1}")
    response = qa_agent.inference(final_input)

    check_len(response)

    return response

## Answer the questions using your pipeline!

Since Colab has usage limit, you might encounter the disconnections. The following code will save your answer for each question. If you have mounted your Google Drive as instructed, you can just rerun the whole notebook to continue your process.

In [108]:
from pathlib import Path

# Fill in your student ID first.
STUDENT_ID = "r12942009"

STUDENT_ID = STUDENT_ID.lower()
with open('./public.txt', 'r') as input_f:
    questions = input_f.readlines()
    questions = [l.strip().split(',')[0] for l in questions]
    for id, question in enumerate(questions, 1):
        if Path(f"./{STUDENT_ID}_{id}.txt").exists():
            continue
        # if id !=1 :
        #   continue
        answer = await pipeline(question)
        answer = answer.replace('\n',' ')
        print(f'{"="*40}')
        print(id, answer)
        with open(f'./{STUDENT_ID}_{id}.txt', 'w') as output_f:
            print(answer, file=output_f)
            pass
        print(f'{"="*40}')

with open('./private.txt', 'r') as input_f:
    questions = input_f.readlines()
    for id, question in enumerate(questions, 31):
        if Path(f"./{STUDENT_ID}_{id}.txt").exists():
          print(id)
          continue
        answer = await pipeline(question)
        answer = answer.replace('\n',' ')
        print(f'{"="*40}')
        print(id, answer)
        with open(f'./{STUDENT_ID}_{id}.txt', 'a') as output_f:
            print(answer, file=output_f)
        print(f'{"="*40}')

     question: 李宏毅（Hung-yi Lee）是台灣著名的人工智慧與機器學習領域的學者，現任教於國立台灣大學電機工程學系。他是台灣大學的教授，也是該校人工智慧研究的領軍人物之一，尤其在機器學習、深度學習和語音處理領域具有深遠的影響。李宏毅教授在學術界的貢獻涵蓋了許多技術領域，特別是在語音處理、自然語言處理方面。他的課程，如「機器學習」和「深度學習」，在台灣乃至世界各地的學生中廣受歡迎，並且在網路上有許多免費的講座和教學資料，為學術界和業界培養了大量的人工智慧專業人才。李教授的研究興趣主要集中在深度學習的各種應用上，包括語音處理、視覺理解、生成模型等。他的學術成就不僅在學術論文上有所體現，還積極推動機器學習技術在實際生活中的應用。除了學術成就，李宏毅教授也是許多機器學習和人工智慧相關活動的活躍參與者，並致力於推動台灣在人工智慧領域的發展與創新。他的教學風格深入淺出，善於將複雜的理論問題以簡單明瞭的方式呈現，因此深受學生的喜愛。那李宏毅在台灣大學開設的《機器學習》 2023 年春季班中，第15個作業的名稱是什麼？

     core_question: 李宏毅教授在《機器學習》2023年春季班中第15個作業的名稱是未知數，因此無法提供具體答案。
     key_words: 李宏毅、人工智慧學者、中華民國大學教授機器学習深度学习
31 根据提供的信息，李宏毅教授在《机器学习》2023年春季班中第15个作业是关于生成式人工智能导论（Generative AI）。
     question: 目前臺灣多數獨立學院皆已升格為大學，公立的獨立學院僅剩一間，請問該獨立學院為何？

     core_question: 該獨立學院是國防管理学院。
     key_words: 獨立學院、公 立大學
32 根据提供的信息，独立学院是中国大陆现存的一种高等学校。它实施本科以上学历教育，是普通高校与国家机构以外社会组织或个人合作利用非财政性经费举办。  公有民辦二級學院（Independent College）是在1999年后出现于中國的獨立院校，主要是由一些原本屬於政府資助創建維持的大專學校轉型而來。這些學府在教育部2003年的《关于规范并加强普通高校以新的机制和模式试办独立学院管理》的通知中被正式確定為“獨立學院”。  公有民辦二級院校（Independen

In [109]:
# Combine the results into one file.
STUDENT_ID = "r12942009"
with open(f'./{STUDENT_ID}.txt', 'w') as output_f:
    for id in range(1,91):
        with open(f'./{STUDENT_ID}_{id}.txt', 'r') as input_f:
            answer = input_f.readline().strip()
            print(answer, file=output_f)

Check Correct

In [110]:
correct = 0
with open(f'./{STUDENT_ID}.txt', 'r') as answer:
    answers = answer.readlines()
with open('./public.txt', 'r') as input_f:
    questions = input_f.readlines()
    questions = [l.strip().split(',')[1] for l in questions]
    for id, question in enumerate(questions, 1):
        sols = question.split('、')
        print(answers[id-1])
        for sol in sols:
          if sol in answers[id-1]:
            correct += 1
            print(id, 'sol->', sol, 'Correct !!!')
            break
        else:
          # print(f'{"="*40}')
          print(id, 'sol->', sol, ' Incorrect')
        print(f'{"="*40}')
print(f'Correct: {correct}/{len(questions)}')

根據提供的資訊，「虎山雄風飛揚」這首歌曲是南投縣光華國小校園背景下的學校之作。

1 sol-> 光華國小 Correct !!!
根據相關資訊，2025年初NCC規定境外郵購自用產品回台的審查費金額為每案新臺幣750元。

2 sol-> 750元 Correct !!!
根據提供的資訊，第一代 iPhone 是由史蒂夫·乔布斯發表。

3 sol-> 史提夫賈伯斯  Incorrect
根據提供的資訊，托福網路測驗 TOEFL iBT 的四個部分分別為：閱讀、聽力、中英文溝通和寫作。考生需要在2.5小時內完成這些項目。  關於達到什麼樣得成績才能申請台灣大學進階英語免修，根據提供的資訊並無明確指出具體分數要求。但是，我們可以參照托福官方網站上的資料。一般而言，大多数台湾大学对 TOEFL iBT 考试成绩有以下基本需求：  * 閱讀：22 分  * 聽力 : 23     中英文溝通:26分以上（口語）     寫作25  這些是大致的要求，但具體情況可能會因為學校和院系而異。因此，我們建議考生查閱各大學入學指南以获取詳細資訊。  最後，請注意，這只是基本需求，並不代表你一定能夠免修進階英語課程。你應該根據自己的實力以及所申報的學校和院系來準備。

4 sol-> 72分或以上  Incorrect
根據問題和相關資訊，我們可以得出以下結論：  觸地 try 可以得到 7 分。  這是因為在橄欖球比賽中，當一名選手持著足球並將其踢入對方的極陣區域時，就會獲得一個達阵（Try），而每個 Try 都可以得分5點。然而，如果該隊伍選擇進行罰門射擊，那麼他們就有機遇再得到另外1或3 分。  因此，觸地 try 可以取得 7 個積極的結果：   * 得到達阵（Try）並獲得了最初五個分數。 + 如果球队选择进行罚门踢，他们可以获得额外的一点积极结果，即得到了額多一點。

5 sol-> 5 Correct !!!
根據提供的資訊，ruvuwa'an位於現今台東縣卑南鄉。

6 sol-> 臺東縣太麻里鄉  Incorrect
根據相關資訊，熊仔的碩班指導教授為李琳山。

7 sol-> 李琳山 Correct !!!
根据提供的信息，迈克尔·法拉第是发现电磁感应定律的人，他于1831年发表了这一研究结果。

8 sol-> Michael Faraday  Incorre